In [1]:
import numpy as np
import os
from abc import abstractmethod
from sklearn.metrics import classification_report

In [2]:
def load_data(folder):
    x_train = np.load(os.path.join(folder, 'x_train.npy'))
    y_train = np.load(os.path.join(folder, 'y_train.npy'))
    x_test = np.load(os.path.join(folder, 'x_test.npy'))
    y_test = np.load(os.path.join(folder, 'y_test.npy'))
    return x_train, y_train, x_test, y_test

In [3]:
def assert_preds_correct(your_preds, sklearn_preds) -> bool:
    return np.abs(your_preds - sklearn_preds).sum() == 0

In [4]:
def assert_probs_correct(your_probs, sklearn_probs) -> bool:
    return np.abs(your_probs - sklearn_probs).mean() < 1e-3

In [5]:
# Не изменяйте код этого класса!
class NaiveBayes:
    def __init__(self, n_classes):
        self.n_classes = n_classes
        self.params = dict()

    # --- PREDICTION ---

    def predict(self, x, return_probs=False):
        """
        x - np.array размерности [N, dim], 
        где N - количество экземпляров данных, 
        dim -размерность одного экземпляра (количество признаков).

        Возвращает np.array размерности [N], содержащий номера классов для
        соответствующих экземпляров.
        """
        preds = []
        for sample in x:
            preds.append(
                self.predict_single(sample, return_probs=return_probs)
            )

        if return_probs:
            return np.array(preds, dtype='float32')

        return np.array(preds, dtype='int32')

    # Совет: вниманительно изучите файл подсказок к данной лабораторной
    # и сопоставьте код с описанной математикой байесовского классификатора.
    def predict_single(self, x, return_probs=False) -> int:
        """
        Делает предсказание для одного экземпляра данных.

        x - np.array размерности dim.

        Возвращает номер класса, которому принадлежит x.
        """
        assert len(
            x.shape) == 1, f'Expected a vector, but received a tensor of shape={x.shape}'
        marginal_prob = self.compute_marginal_probability(
            x)  # P(x) - безусловная вероятность появления x

        probs = []
        for c in range(self.n_classes):                 # c - номер класса
            # P(c) - априорная вероятность (вероятность появления класса)
            prior = self.compute_prior(c)
            # P(x|c) - вероятность появления x в предположении, что он принаждлежит c
            likelihood = self.compute_likelihood(x, c)

            # Используем теорему Байесса для просчёта условной вероятности P(c|x)
            # P(c|x) = P(c) * P(x|c) / P(x)
            prob = prior * likelihood / marginal_prob
            probs.append(prob)

        if return_probs:
            return probs

        return np.argmax(probs)

    # Вычисляет P(x) - безусловная вероятность появления x.
    @abstractmethod
    def compute_marginal_probability(self, x) -> float:
        pass

    # Вычисляет P(c) - априорная вероятность появления класса c.
    @abstractmethod
    def compute_prior(self, c) -> float:
        pass

    # Вычисляет P(x|c) - вероятность наблюдения экземпляра x в предположении, что он принаждлежит c.
    @abstractmethod
    def compute_likelihood(self, x, c) -> float:
        pass

    # --- FITTING ---

    def fit(self, x, y):
        self._estimate_prior(y)
        self._estimate_params(x, y)

    @abstractmethod
    def _estimate_prior(self, y):
        pass

    @abstractmethod
    def _estimate_params(self, x, y):
        pass

## 1. Наивный классификатор Байеса: гауссово распределение


Напишите недостающий код, создайте и обучите модель.

Пункты оценки:

1. совпадение предсказанных классов с оными у модели sklearn. Для проверки совпадения используйте функцию `assert_preds_correct`.
2. совпадение значений предсказанных вероятностей принадлежности классами с оными у модели sklearn. Значения вероятностей считаются равными, если функция `assert_probs_correct` возвращает True.


In [6]:
x_train, y_train, x_test, y_test = load_data('gauss')

In [7]:
# P(C_k|x) = P(x|theta) * P(C_k) / P(x)

class NaiveGauss(NaiveBayes):

    def compute_marginal_probability(self, x) -> float:
        # Вычисляет P(x)
        # Для просчёта безусловной вероятности используйте
        # методы compute_prior и compute_likelihood.
        # Напишите свой код здесь
        p_a = 0
        for c, _ in enumerate(self.params['stats']):
            p_a += self.compute_prior(c) * self.compute_likelihood(x, c)
        return p_a

    def compute_prior(self, c: int) -> float:
        # Вычисляет P(c)
        assert abs(sum(self.params['prior']) - 1.0) < 1e-3, \
            f"Sum of prior probabilities must be equal to 1, but is {sum(self.params['prior'])}"
        assert c < self.n_classes, f'Class index must be < {self.n_classes}, but received {c}.'
        return self.params['prior'][c]

    @staticmethod
    def gaussian_func(x, avg, disp):
        return (
            (1 / np.sqrt(2 * np.pi * np.pow(disp, 2))) *
            np.exp(
                (-1) * (np.pow(x - avg, 2) /
                        (2 * np.pow(disp, 2)))
            )
        )

    def compute_likelihood(self, x, c: int) -> float:
        # Вычисляет P(x|c)
        assert c < self.n_classes, f'Class index must be < {self.n_classes}, but received {c}.'
        final_likelihood = None
        for x_pos, feature_stat in enumerate(self.params['stats'][c]):
            avg, disp = feature_stat
            if final_likelihood is None:
                final_likelihood = self.gaussian_func(x[x_pos], avg, disp)
            else:
                final_likelihood *= self.gaussian_func(x[x_pos], avg, disp)
        return final_likelihood

    # --- FITTING ---

    def _estimate_prior(self, y: np.ndarray) -> None:
        total = y.shape[0]
        prior_probs = []
        for cls in sorted(set(y)):
            prior_probs.append(len(y[np.where(y == cls)]) / total)
        self.params['prior'] = prior_probs

    def _estimate_params(
        self,
        x: np.ndarray,
        y: np.ndarray
    ) -> None:
        classes_feature_stats = []
        for cls in set(y):
            subset = x[np.where(y == cls)]
            cls_feture_stats = []
            for x_features in range(subset.shape[1]):
                feature_slice = subset[:, x_features]
                cls_feture_stats.append(
                    (np.mean(feature_slice), np.std(feature_slice)))
            classes_feature_stats.append(tuple(cls_feture_stats))
        self.params['stats'] = tuple(classes_feature_stats)

In [8]:
ng = NaiveGauss(len(set(y_train)))
ng.fit(x_train, y_train)

In [9]:
# Создайте и обучите модель
ng = NaiveGauss(len(set(y_train)))
ng.fit(x_train.copy(), y_train.copy())

In [10]:
# Оцените качество модели
print(classification_report(y_test, ng.predict(x_test)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       1.00      1.00      1.00        23
           2       1.00      1.00      1.00        19

    accuracy                           1.00        60
   macro avg       1.00      1.00      1.00        60
weighted avg       1.00      1.00      1.00        60



In [11]:
# Сравните вашу модель с аналогом sklearn (GaussianNB)
from sklearn.naive_bayes import GaussianNB
skng = GaussianNB()
skng.fit(x_train, y_train)

print(f'{assert_preds_correct(ng.predict(x_test), skng.predict(x_test))=}')
print(f'{assert_probs_correct(ng.predict(x_test, True), skng.predict_proba(x_test))=}')

assert_preds_correct(ng.predict(x_test), skng.predict(x_test))=np.True_
assert_probs_correct(ng.predict(x_test, True), skng.predict_proba(x_test))=np.True_


## 2. Доп. задания (любое на выбор, опционально)


### 2.1 Упрощение наивного классификатора Байеса для гауссова распределения


Уберите из класса NaiveBayes 'лишние' вычисления и удалите код, что соответствует этим вычислениям. Под 'лишним' подразумеваются вещи, что не влияют на итоговое решение о принадлежности классу (значения вероятностей при этом могу стать некорректными, но в данном задании это допустимо).

Напишите в клетке ниже код упрощенного 'классификатора Гаусса' и убедитесь, что его ответы (не значения вероятностей) совпадают с ответами классификатора из задания 1. Для сравнения ответов используйте функцию `assert_preds_correct`.

Указание: работайте в предположении, что классы равновероятны.

Подсказка: упростить необходимо метод `predict_single`.


In [12]:
# Напишите обновленный код модели здесь

# Не изменяйте код этого класса!
class NaiveBayes:
    def __init__(self, n_classes):
        self.n_classes = n_classes
        self.params = dict()

    # --- PREDICTION ---

    def predict(self, x, return_probs=False):
        """
        x - np.array размерности [N, dim], 
        где N - количество экземпляров данных, 
        dim -размерность одного экземпляра (количество признаков).

        Возвращает np.array размерности [N], содержащий номера классов для
        соответствующих экземпляров.
        """
        preds = []
        for sample in x:
            preds.append(
                self.predict_single(sample, return_probs=return_probs)
            )

        if return_probs:
            return np.array(preds, dtype='float32')

        return np.array(preds, dtype='int32')

    # Совет: вниманительно изучите файл подсказок к данной лабораторной
    # и сопоставьте код с описанной математикой байесовского классификатора.
    def predict_single(self, x, return_probs=False) -> int:
        """
        Делает предсказание для одного экземпляра данных.

        x - np.array размерности dim.

        Возвращает номер класса, которому принадлежит x.
        """
        assert len(
            x.shape) == 1, f'Expected a vector, but received a tensor of shape={x.shape}'

        probs = []
        for c in range(self.n_classes):                 # c - номер класса
            # P(c) - априорная вероятность (вероятность появления класса)
            # P(x|c) - вероятность появления x в предположении, что он принаждлежит c
            likelihood = self.compute_likelihood(x, c)

            # Используем теорему Байесса для просчёта условной вероятности P(c|x)
            # P(c|x) = P(c) * P(x|c) / P(x)
            prob = likelihood
            probs.append(prob)

        if return_probs:
            return probs

        return np.argmax(probs)

    # Вычисляет P(x) - безусловная вероятность появления x.
    @abstractmethod
    def compute_marginal_probability(self, x) -> float:
        pass

    # Вычисляет P(c) - априорная вероятность появления класса c.
    @abstractmethod
    def compute_prior(self, c) -> float:
        pass

    # Вычисляет P(x|c) - вероятность наблюдения экземпляра x в предположении, что он принаждлежит c.
    @abstractmethod
    def compute_likelihood(self, x, c) -> float:
        pass

    # --- FITTING ---

    def fit(self, x, y):
        self._estimate_prior(y)
        self._estimate_params(x, y)

    @abstractmethod
    def _estimate_prior(self, y):
        pass

    @abstractmethod
    def _estimate_params(self, x, y):
        pass


class NaiveGauss(NaiveBayes):

    def compute_marginal_probability(self, x) -> float:
        # Вычисляет P(x)
        # Для просчёта безусловной вероятности используйте
        # методы compute_prior и compute_likelihood.
        # Напишите свой код здесь
        p_a = 0
        for c, _ in enumerate(self.params['stats']):
            p_a += self.compute_prior(c) * self.compute_likelihood(x, c)
        return p_a

    def compute_prior(self, c: int) -> float:
        # Вычисляет P(c)
        assert abs(sum(self.params['prior']) - 1.0) < 1e-3, \
            f"Sum of prior probabilities must be equal to 1, but is {sum(self.params['prior'])}"
        assert c < self.n_classes, f'Class index must be < {self.n_classes}, but received {c}.'
        return self.params['prior'][c]

    @staticmethod
    def gaussian_func(x, avg, disp):
        return (
            (1 / np.sqrt(2 * np.pi * np.pow(disp, 2))) *
            np.exp(
                (-1) * (np.pow(x - avg, 2) /
                        (2 * np.pow(disp, 2)))
            )
        )

    def compute_likelihood(self, x, c: int) -> float:
        # Вычисляет P(x|c)
        assert c < self.n_classes, f'Class index must be < {self.n_classes}, but received {c}.'
        final_likelihood = None
        for x_pos, feature_stat in enumerate(self.params['stats'][c]):
            avg, disp = feature_stat
            if final_likelihood is None:
                final_likelihood = self.gaussian_func(x[x_pos], avg, disp)
            else:
                final_likelihood *= self.gaussian_func(x[x_pos], avg, disp)
        return final_likelihood

    # --- FITTING ---

    def _estimate_prior(self, y: np.ndarray) -> None:
        total = y.shape[0]
        prior_probs = []
        for cls in sorted(set(y)):
            prior_probs.append(len(y[np.where(y == cls)]) / total)
        self.params['prior'] = prior_probs

    def _estimate_params(
        self,
        x: np.ndarray,
        y: np.ndarray
    ) -> None:
        classes_feature_stats = []
        for cls in set(y):
            subset = x[np.where(y == cls)]
            cls_feture_stats = []
            for x_features in range(subset.shape[1]):
                feature_slice = subset[:, x_features]
                cls_feture_stats.append(
                    (np.mean(feature_slice), np.std(feature_slice)))
            classes_feature_stats.append(tuple(cls_feture_stats))
        self.params['stats'] = tuple(classes_feature_stats)

In [13]:
# Создайте и обучите модель
ng1 = NaiveGauss(len(set(y_train)))
ng1.fit(x_train, y_train)

In [14]:
# Оцените качество модели
print(classification_report(y_test, ng1.predict(x_test)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       1.00      1.00      1.00        23
           2       1.00      1.00      1.00        19

    accuracy                           1.00        60
   macro avg       1.00      1.00      1.00        60
weighted avg       1.00      1.00      1.00        60



In [15]:
# Сравните вашу модель с моделью из задания 1
print(f'{assert_preds_correct(ng1.predict(x_test), ng.predict(x_test))=}')
print(f'{assert_probs_correct(ng1.predict(x_test, True), ng.predict(x_test, True))=}')

assert_preds_correct(ng1.predict(x_test), ng.predict(x_test))=np.True_
assert_probs_correct(ng1.predict(x_test, True), ng.predict(x_test, True))=np.False_


### Объясните в комментариях к этой клетке суть проделанных изменений: почему удаленный код является лишним?

1. Убрано умножение на prior

   Исходя из предположения, что каждый класс равновероятен, следует что априорная вероятность для каждого класса будет одинакова.

   Это в свою очередь означает, что на конечную вероятность данная веростность никакого влияния оказывать не будет, соотетственно её можно исключить из подсчёта, максимальная вероятность принадлежности останется у того же класса

2. Убрано деление на marginal_prob

   Эта операция выступала в качестве нормализации данных и приведении к формату [0..1]

   В нашем же случае необходимо предсказать класс, а не его вероятность, соотетственно что без деления, что с делением, мы выбираем максимальное просчитанное значение, на основе чего и делаем вывод - что это за класс.
